In [ ]:
import sys


In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D

Using TensorFlow backend.


In [2]:
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils

In [3]:
import tensorflow as tf
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import skvideo.io
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import preprocessing

/home/misha/tensorflow/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)
/home/misha/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


### Image specification

In [ ]:
img_rows,img_cols,img_depth=60,80,15

### Training data

In [ ]:
X_tr = [] # variable to store entire dataset

In [ ]:
directory = '/home/misha/Documents/Thesis/youtube_dataset/'
count = 0
for file in os.listdir(directory):
    if file[0] == str(0):
        print(file)
        count += 1
        
print(count)

In [ ]:
directory = '/home/misha/Documents/Thesis/youtube_dataset/'
frames_count = 0

for file in os.listdir(directory):
    if file[0] == str(0):
        print(file)
        videodata = skvideo.io.vreader(directory + file)
        for frame in videodata:    
            frames_count += 1
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            res_img = cv2.resize(frame, (img_cols, img_rows), interpolation=cv2.INTER_AREA)
            X_tr.append(res_img)
            if frames_count > 10000:
                break
        else:
            continue
        break
    
zero_frames = frames_count    
print('Zero class:', frames_count)





In [ ]:
frames_count = 0
for file in os.listdir(directory):
    if file[0] == str(1):
        print(file)
        videodata = skvideo.io.vreader(directory + file)
        for frame in videodata:    
            frames_count += 1
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            res_img = cv2.resize(frame, (img_cols, img_rows), interpolation=cv2.INTER_AREA)
            X_tr.append(res_img)
            if frames_count > 10000:
                break
        else:
            continue
        break
        
print('One class', frames_count)

In [ ]:
len(X_tr)

### Open files

### Reading accident class training

In [ ]:
# frames numbers hardcoded just for debug. Normally they are in .csv file.
# addFrames(X_tr, 3248, 3283)
# addFrames(X_tr, 3492, 3527)

plt.imshow(X_tr[-1],cmap='gray')
plt.show()


In [ ]:
# frames = open(filename + '.csv', 'r')
# ret, frame = cap.read()
# frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY )
plt.imshow(img, cmap='gray')
plt.show()
print(img)

##### Image resize

In [ ]:
# now resize in reading dunction
plt.imshow(X_tr[630], cmap='gray')
plt.show()

### Reading all frames classes (takes long time)

In [ ]:
# X_tr_array.shape

### CNN Training parameters

In [ ]:
# Importaint step
X_tr_array = np.array(X_tr)

In [ ]:
norm_ipt = np.rollaxis(np.rollaxis(norm_input,2,0),2,0)
print(norm_ipt.shape)
acc_ipt = np.rollaxis(np.rollaxis(acc_input,2,0),2,0)
print(acc_ipt.shape)



In [ ]:
# convert to array
X_tr_norm = np.array(norm_ipt)
X_tr_acc = np.array(acc_ipt)

num_samples_norm = len(X_tr_norm)
num_samples_acc = len(X_tr_acc)
print(num_samples_norm)
print(num_samples_acc)

In [ ]:
print(X_tr_array.shape)

In [ ]:

X_tr_array = np.expand_dims(X_tr_array, axis=4)
print(X_tr_array.shape)

##### Assign label to each class

In [ ]:
num_samples = len(X_tr_array)
# train_set = np.zeros(num_samples, 1, img_rows,img_cols,img_depth))
label = np.ones((num_samples,),dtype = int)
label[0:zero_frames] = 0
label[zero_frames:] = 1

train_data = [X_tr_array,label]

(X_train, y_train) = (train_data[0],train_data[1])
print('X_Train shape:', X_train.shape)

In [ ]:
print(len(y_train))

In [ ]:
print(num_samples, img_rows,img_cols,img_depth)

i = 0
while i < num_samples:
    


In [ ]:
train_set = np.zeros((num_samples,img_rows,img_cols,img_depth))
# train_set = np.zeros((20449,img_rows,img_cols,img_depth))
patch_size = 15

# for h in range(num_samples):
#     train_set[h][0][:][:][:]=X_train[h,:,:,:]


In [ ]:
print(train_set.shape, 'train samples')

##### CNN training parameters

In [5]:
batch_size = 2
nb_classes = 2 
nb_epoch = 100


##### convert class vectors to binary class matrices

In [ ]:
Y_train = np_utils.to_categorical(y_train, nb_classes)

##### number of convolutional filters to use at each layer

In [6]:
nb_filters = [32, 32]

##### level of pooling to perform at each layer (POOL x POOL)

In [7]:
nb_pool = [2, 2]

##### Level of convolution to perform at each layer (CONV x CONV)

In [8]:
nb_conv = [3,3]

### Pre-processing

In [ ]:
train_set = train_set.astype('float32')

train_set -= np.mean(train_set)

train_set /= np.max(train_set)

In [ ]:
print(img_rows, img_cols, img_depth)

### Define model

In [9]:
model = Sequential()
model.add(Convolution3D(16, (4, 4, 3),
input_shape=(60, 80, 15, 1), activation='relu'))

model.add(MaxPooling3D(pool_size=(nb_pool[0], nb_pool[0], nb_pool[0])))

model.add(Convolution3D(32, (3, 3, 3), activation='relu'))

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Convolution3D(64, (3, 3, 2), activation='relu'))

model.add(MaxPooling3D(pool_size=(2, 2, 1)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128, init='normal', activation='relu'))

# model.add(Dropout(0.5))

model.add(Dense(nb_classes,init='normal'))

model.add(Activation('softmax'))

# model.optimizer.lr.assign(0.1)

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", kernel_initializer="normal")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="normal")`


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 57, 77, 13, 16)    784       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 38, 6, 16)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 36, 4, 32)     13856     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 13, 18, 2, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 11, 16, 1, 64)     36928     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 5, 8, 1, 64)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 8, 1, 64)       0         
__________

In [ ]:
# Split the data
# X_train_new, X_val_new, y_train_new,y_val_new =  train_test_split(train_set, Y_train, test_size=0.2, random_state=4)
X_train_new, X_val_new, y_train_new,y_val_new =  train_test_split(train_set, Y_train, test_size=0.2, random_state=4)

In [ ]:
X_train_new = np.expand_dims(X_train_new, axis=4)
print(X_train_new.shape)

X_val_new = np.expand_dims(X_val_new, axis=4)
print(X_val_new.shape)

#### Training

In [ ]:
hist = model.fit(X_train_new, y_train_new, validation_data=(X_val_new,y_val_new),
          batch_size=batch_size,nb_epoch = nb_epoch,shuffle=True)

### Split the data

In [ ]:
plt.figure(figsize=(30,20))
for i in range(0,10):
    img = acc_frames[i]
    plt.subplot(1,10,i + 1)
    plt.imshow(img,cmap='gray')
    
    img = normal_frames[i]
    plt.subplot(2,10,i + 1)
    plt.imshow(img,cmap='gray')
plt.show()

### Train the model

In [ ]:
frame = acc_frames[-1]
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
value = 1
hsv[:,:,2] += value
hsv = cv2.cvtColor(frame, cv2.COLOR_HSV2RGB)

plt.imshow(hsv)
plt.show()

In [ ]:
score

### Evaluate the model

In [ ]:
score = model.evaluate(X_val_new, y_val_new, batch_size=batch_size)
print('Test score:', score[0])
print('Test accuracy:', score[1]) 


In [ ]:
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']

plt.plot(train_loss)
plt.plot(val_loss)
plt.plot(train_acc)
plt.plot(val_acc)
plt.xlabel('num of Epochs')
plt.grid(True)
plt.legend(['train_loss','val_loss','train_acc', 'val_acc'])
plt.style.use(['classic'])
plt.show()

### Plots the result

In [ ]:
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(10)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print(plt.style.available) # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])